# Introduction

In this tutorial, we will show how to use PyTupli to set up an efficient pipeline for offline reinforcement learning (RL) for a custom environment. This includes
- creating a benchmark and uploading it to an instance of a TupliStorage,
- re-loading this benchmark from the storage, 
- recording RL tuples of (state, action, reward, done) for this benchmark and uploading them to the storage, 
- creating a dataset from the stored episodes, and
- training an offline RL agent using d3rlpy.

You can skip the last part, but if you want to try that, you have to install the d3rlpy library using `pip install d3rlpy`.

In [1]:
import io
import os
import tempfile
import math
from typing import Optional

import numpy as np
import pandas as pd

from gymnasium import spaces
from gymnasium.envs.classic_control import utils, MountainCarEnv
from gymnasium.wrappers import TimeLimit

import d3rlpy
from d3rlpy.algos import DiscreteCQLConfig
from d3rlpy.dataset import MDPDataset

from pytupli.benchmark import TupliEnvWrapper
from pytupli.storage import TupliAPIClient, TupliStorage, FileStorage
from gymnasium import Env
from pytupli.schema import ArtifactMetadata, FilterEQ, EpisodeMetadataCallback
from pytupli.dataset import TupliDataset, NumpyTupleParser

/home/hannah/anaconda3/envs/pytupli_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-07-28 15:45.23 [info     ] Register Shimmy environments. 


PyTupli has two storage options: A local FileStorage and using MongoDB as a backend in the TupliAPIClient. You can run this notebook with both storage types by adjusting the flag below. If you want to use the TupliAPIClient, follow the instructions in the Readme to start the application.

In [2]:
STORAGE_FLAG = 'api'  # "api"

### Creating a Custom Environment
We will use the MountainCar example from gymnasium with a small modification: The cart is slowed down by wind in the horizontal direction. We load the wind data from a csv file.

In [3]:
class CustomMountainCarEnv(MountainCarEnv):
    """
    ## Description

    The Mountain Car MDP is a deterministic MDP that consists of a car placed stochastically
    at the bottom of a sinusoidal valley, with the only possible actions being the accelerations
    that can be applied to the car in either direction. The goal of the MDP is to strategically
    accelerate the car to reach the goal state on top of the right hill. There are two versions
    of the mountain car domain in gymnasium: one with discrete actions and one with continuous.
    This version is the one with discrete actions.

    This MDP first appeared in [Andrew Moore's PhD Thesis (1990)](https://www.cl.cam.ac.uk/techreports/UCAM-CL-TR-209.pdf)

    ```
    @TECHREPORT{Moore90efficientmemory-based,
        author = {Andrew William Moore},
        title = {Efficient Memory-based Learning for Robot Control},
        institution = {University of Cambridge},
        year = {1990}
    }
    ```

    ## Observation Space

    The observation is a `ndarray` with shape `(2,)` where the elements correspond to the following:

    | Num | Observation                          | Min   | Max  | Unit         |
    |-----|--------------------------------------|-------|------|--------------|
    | 0   | position of the car along the x-axis | -1.2  | 0.6  | position (m) |
    | 1   | velocity of the car                  | -0.07 | 0.07 | velocity (v) |

    ## Action Space

    There are 3 discrete deterministic actions:

    - 0: Accelerate to the left
    - 1: Don't accelerate
    - 2: Accelerate to the right

    ## Transition Dynamics:

    Given an action, the mountain car follows the following transition dynamics:

    *velocity<sub>t+1</sub> = velocity<sub>t</sub> + (action - 1) * force - cos(3 * position<sub>t</sub>) * gravity*

    *position<sub>t+1</sub> = position<sub>t</sub> + velocity<sub>t+1</sub>*

    where force = 0.001 and gravity = 0.0025. The collisions at either end are inelastic with the velocity set to 0
    upon collision with the wall. The position is clipped to the range `[-1.2, 0.6]` and
    velocity is clipped to the range `[-0.07, 0.07]`.

    ## Reward:

    The goal is to reach the flag placed on top of the right hill as quickly as possible, as such the agent is
    penalised with a reward of -1 for each timestep.

    ## Starting State

    The position of the car is assigned a uniform random value in *[-0.6 , -0.4]*.
    The starting velocity of the car is always assigned to 0.

    ## Episode End

    The episode ends if either of the following happens:
    1. Termination: The position of the car is greater than or equal to 0.5 (the goal position on top of the right hill)
    2. Truncation: The length of the episode is 200.

    ## Arguments

    Mountain Car has two parameters for `gymnasium.make` with `render_mode` and `goal_velocity`.
    On reset, the `options` parameter allows the user to change the bounds used to determine the new random state.

    ```python
    >>> import gymnasium as gym
    >>> env = gym.make("MountainCar-v0", render_mode="rgb_array", goal_velocity=0.1)  # default goal_velocity=0
    >>> env
    <TimeLimit<OrderEnforcing<PassiveEnvChecker<MountainCarEnv<MountainCar-v0>>>>>
    >>> env.reset(seed=123, options={"x_init": np.pi/2, "y_init": 0.5})  # default x_init=np.pi, y_init=1.0
    (array([-0.46352962,  0.        ], dtype=float32), {})

    ```

    ## Version History

    * v0: Initial versions release
    """

    metadata = {
        'render_modes': ['human', 'rgb_array'],
        'render_fps': 30,
    }

    def __init__(self, data_path: str, render_mode: Optional[str] = None, goal_velocity=0):
        self.min_position = -1.2
        self.max_position = 0.6
        self.max_speed = 0.07
        self.goal_position = 0.5
        self.goal_velocity = goal_velocity
        self.current_step = 0
        self.data = pd.read_csv(data_path, index_col=0, header=None) * 0.01

        self.force = 0.001
        self.gravity = 0.0025

        self.low = np.array([self.min_position, -self.max_speed], dtype=np.float32)
        self.high = np.array([self.max_position, self.max_speed], dtype=np.float32)

        self.render_mode = render_mode

        self.screen_width = 600
        self.screen_height = 400
        self.screen = None
        self.clock = None
        self.isopen = True

        self.action_space = spaces.Discrete(3)
        self.observation_space = spaces.Box(self.low, self.high, dtype=np.float32)

    def step(self, action: int):
        assert self.action_space.contains(action), f'{action!r} ({type(action)}) invalid'

        position, velocity = self.state
        velocity += (
            (action - 1) * self.force
            + math.cos(3 * position) * (-self.gravity)
            - self.data.loc[self.current_step].to_numpy().flatten()[0] * math.cos(position)
        )
        velocity = np.clip(velocity, -self.max_speed, self.max_speed)
        position += velocity
        position = np.clip(position, self.min_position, self.max_position)
        if position == self.min_position and velocity < 0:
            velocity = 0

        terminated = bool(position >= self.goal_position and velocity >= self.goal_velocity)
        reward = -1.0
        self.current_step += 1

        self.state = (position, velocity)
        if self.render_mode == 'human':
            self.render()
        # truncation=False as the time limit is handled by the `TimeLimit` wrapper added during `make`
        return np.array(self.state, dtype=np.float32), reward, terminated, False, {}

    def reset(
        self,
        *,
        seed: Optional[int] = None,
        options: Optional[dict] = None,
    ):
        super().reset(seed=seed)
        # Note that if you use custom reset bounds, it may lead to out-of-bound
        # state/observations.
        self.current_step = 0
        low, high = utils.maybe_parse_reset_bounds(options, -0.6, -0.4)
        self.state = np.array([self.np_random.uniform(low=low, high=high), 0])

        if self.render_mode == 'human':
            self.render()
        return np.array(self.state, dtype=np.float32), {}

### Serialize Environment for Upload
As a next step, we want to upload our environment to our storage using PyTupli. For this, we will detach the csv file from the environment, upload it seperately, and replace the data attribute in the environment with the id of the stored object. This allows us to re-use artifacts such as csv files in multiple benchmarks. For example, consider a case where you only want to change one parameter within the environment, e.g., the maximum speed. You would have to create a new benchmark, but could re-use the csv file! PyTupli automatically recognizes such duplicates. 

To separate the csv file, we have to subclass the TupliEnvWrapper class and overwrite the `_serialize()` and `_deserialize()` members. The TupliEnvWrapper is essentially a gymnasium wrapper that records RL tuples in the `step()` function, but it has a lot of extra functionalities for interacting with the storage.

In [4]:
class MyTupliEnvWrapper(TupliEnvWrapper):
    def _serialize(self, env) -> Env:
        related_data_sources = []
        ds = env.unwrapped.data
        metadata = ArtifactMetadata(name='test')
        data_kwargs = {'header': None}
        try:
            content = ds.to_csv(encoding='utf-8', **data_kwargs)
            content = content.encode(encoding='utf-8')
        except Exception as e:
            raise ValueError(f'Failed to serialize data source: {e}')

        ds_storage_metadata = self.storage.store_artifact(artifact=content, metadata=metadata)
        related_data_sources.append(ds_storage_metadata.id)
        setattr(env.unwrapped, 'data', ds_storage_metadata.id)
        return env, related_data_sources

    @classmethod
    def _deserialize(cls, env: Env, storage: TupliStorage) -> Env:
        data_kwargs = {'header': None, 'index_col': 0}
        ds = storage.load_artifact(env.unwrapped.data)
        ds = ds.decode('utf-8')
        d = io.StringIO(ds)
        df = pd.read_csv(d, **data_kwargs)

        env.unwrapped.data = df
        return env

In [5]:
# which storag to use
if STORAGE_FLAG == 'api':
    storage = TupliAPIClient()
elif STORAGE_FLAG == 'file':
    storage = FileStorage()
else:
    raise ValueError(f"Unknown storage flag: {STORAGE_FLAG}. Has to be 'api' or 'file'.")

In [6]:
# instantiate the environment
max_eps_length = 999
data_path = '/home/hannah/Documents/Code/pytupli/docs/source/tutorials/data/wind_data.csv'
env = TimeLimit(
    CustomMountainCarEnv(render_mode=None, data_path=data_path), max_episode_steps=max_eps_length
)
# Now we can create the benchmark
tupli_env = MyTupliEnvWrapper(env, storage=storage)

### Uploading and Downloading Benchmarks
We will now upload the benchmark and download it again.

In [7]:
tupli_env.store(name='mountain-car-v0', description='Mountain Car v0 benchmark')

Let us list the uploaded benchmarks:

In [8]:
%system
!pytupli list_benchmarks

id                                created_by    is_public    created_at                  hash                                                              metadata
--------------------------------  ------------  -----------  --------------------------  ----------------------------------------------------------------  -------------------------------------------------------------------------------------------------------------------------
9ab24bb4e96c4b38bf0c3577bdb4e4bb  hannah        ❌           2025-07-10T13:37:28.852619  a23a5fdeafb580c2d170f19da7ce4969164d8e458c6eeef2494ad71ff5c340c1  {'name': 'test_env', 'description': '', 'difficulty': None, 'version': None, 'extra': {}}
411704fb2dd34f50982817141264840b  hannah        ❌           2025-07-10T13:48:14.966254  6837628829fbb513e75eae4e0f33ad322f310780731c2308ab90fc4218fb8d47  {'name': 'second_test_env', 'description': '', 'difficulty': None, 'version': None, 'extra': {}}
989f723675fc4b85ab19c1621a96f9dc  hannah        ❌           2025

As a next step, we show how to download the benchmark. Note that this is only for demonstration purposes! When loading the benchmark, we can pass a callback that will later be used to add metadate to recorded episodes. We provide a simple example of such a function.

In [9]:
class MyCallback(EpisodeMetadataCallback):
    def __init__(self):
        super().__init__()
        # we will compute the cumulative reward for an episode
        self.cum_reward = 0
        # Furthermore, we want to store the fact that the episode was not an expert episode
        self.is_expert = False
    def reset(self):
        # we will compute the cumulative reward for an episode
        self.cum_reward = 0
    def __call__(self, tuple):
        self.cum_reward += tuple.reward
        return {"cum_eps_reward": [self.cum_reward], "is_expert": self.is_expert}


In [10]:
loaded_tupli_env = MyTupliEnvWrapper.load(storage=storage, benchmark_id=tupli_env.id, metadata_callback=MyCallback())

### Recording Episodes for Offline RL Training
The TupliEnvWrapper wrapper allows us to record all interactions with the custom environment to the storage in form of tuples (state, action, reward, terminal, timeout). This can then be used for training an offline RL agent for this environment using any offline RL library. For simplicity, we will use a random policy to generate the data.

In [11]:
# For reproducibility when generating episodes
np.random.seed(42)
obs, info = loaded_tupli_env.reset(seed=42)

for step in range(2000):
    action = np.int64(np.random.randint(low=0, high=3))
    obs, reward, done, truncated, info = loaded_tupli_env.step(action)
    if done or truncated:
        print(f'Episode finished after {step + 1} timesteps')
        obs, info = loaded_tupli_env.reset()

Episode finished after 999 timesteps
Episode finished after 1998 timesteps


### Downloading Episodes for a Benchmark
Next, let us download all episodes that have been recorded for our benchmark. For this, we create a TupliDataset using a filter with the id of the benchmark.

In [12]:
# Create dataset
mdp_dataset = TupliDataset(storage=storage).with_benchmark_filter(
    FilterEQ(key='id', value=loaded_tupli_env.id)
)
mdp_dataset.load()

We can show the contents of the dataset using the `preview()` method:

In [13]:
mdp_dataset.preview()

[EpisodeItem(id='8236cd52226b44cdb9ed453d180ce4fa', created_by='hannah', is_public=False, created_at='2025-07-28T15:45:24.757499', benchmark_id='4b076cdedc4d433ea4da6d2801c05ca0', metadata={'cum_eps_reward': [-999.0], 'is_expert': False}, n_tuples=999, terminated=False, timeout=True, tuples=[RLTuple(state=[-0.5113905072212219, 0.0008338160114362836], action=2, reward=-1.0, info={}, terminal=False, timeout=False), RLTuple(state=[-0.5116723775863647, -0.0002818846551235765], action=0, reward=-1.0, info={}, terminal=False, timeout=False), RLTuple(state=[-0.5110931396484375, 0.0005792493466287851], action=2, reward=-1.0, info={}, terminal=False, timeout=False), RLTuple(state=[-0.5096522569656372, 0.0014408753486350179], action=2, reward=-1.0, info={}, terminal=False, timeout=False), RLTuple(state=[-0.5093242526054382, 0.000327992340316996], action=0, reward=-1.0, info={}, terminal=False, timeout=False), RLTuple(state=[-0.5101639032363892, -0.0008396401535719633], action=0, reward=-1.0, inf

### Training an Offline RL Agent
Finally, we use d3rlpy to train an offline RL agent on this environment. Note that we do not train it to convergence, we only show how to get from a PyTupli dataset to actually doing offline RL! Our TupliDataset has a method for converting all episodes into numpy arrays for states, actions, rewards, terminals, and timeouts. This can be customized if other output formats are required. Using these arrays, we can then create an `MDPDataset`, which is the required input format for all d3rlpy algorithms. 

In [14]:
obs, act, rew, terminal, truncated = mdp_dataset.convert_to_tensors(parser=NumpyTupleParser)
# create d3rlpy dataset
d3rlpy_dataset = MDPDataset(
    observations=obs, actions=act, rewards=rew, terminals=terminal, timeouts=truncated
)

2025-07-28 15:45.25 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('float32')], shape=[(1,)]) observation_signature=Signature(dtype=[dtype('float32')], shape=[(2,)]) reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)])
2025-07-28 15:45.25 [info     ] Action-space has been automatically determined. action_space=<ActionSpace.DISCRETE: 2>
2025-07-28 15:45.25 [info     ] Action size has been automatically determined. action_size=3


Finally, let us show that training an agent using conservative Q-learning (CQL) works with this data:

In [15]:
# algorithm for offline training: CQL from d3rlpy
d3rlpy.seed(1)  # for reproducibility
algo = DiscreteCQLConfig(batch_size=64, alpha=2.0, target_update_interval=1000).create(device='cpu')
# train
algo.fit(dataset=d3rlpy_dataset, n_steps=10000, n_steps_per_epoch=100)

2025-07-28 15:45.25 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float32')], shape=[(2,)]), action_signature=Signature(dtype=[dtype('float32')], shape=[(1,)]), reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)]), action_space=<ActionSpace.DISCRETE: 2>, action_size=3)
2025-07-28 15:45.25 [debug    ] Building models...            
2025-07-28 15:45.25 [debug    ] Models have been built.       
2025-07-28 15:45.25 [info     ] Directory is created at d3rlpy_logs/DiscreteCQL_20250728154525
2025-07-28 15:45.25 [info     ] Parameters                     params={'observation_shape': [2], 'action_size': 3, 'config': {'type': 'discrete_cql', 'params': {'batch_size': 64, 'gamma': 0.99, 'observation_scaler': {'type': 'none', 'params': {}}, 'action_scaler': {'type': 'none', 'params': {}}, 'reward_scaler': {'type': 'none', 'params': {}}, 'compile_graph': False, 'learning_rate': 6.25e-05, 'optim_factory': {'type': 'ada

Epoch 1/100: 100%|██████████| 100/100 [00:00<00:00, 377.93it/s, loss=2.41, td_loss=0.213, conservative_loss=1.1]

2025-07-28 15:45.25 [info     ] DiscreteCQL_20250728154525: epoch=1 step=100 epoch=1 metrics={'time_sample_batch': 0.0005567145347595215, 'time_algorithm_update': 0.0020180344581604004, 'loss': 2.3966327929496765, 'td_loss': 0.19853577110916376, 'conservative_loss': 1.09904851436615, 'time_step': 0.00263277530670166} step=100
2025-07-28 15:45.25 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteCQL_20250728154525/model_100.d3



Epoch 2/100: 100%|██████████| 100/100 [00:00<00:00, 439.61it/s, loss=2.21, td_loss=0.016, conservative_loss=1.1]

2025-07-28 15:45.25 [info     ] DiscreteCQL_20250728154525: epoch=2 step=200 epoch=2 metrics={'time_sample_batch': 0.0004550933837890625, 'time_algorithm_update': 0.00175459623336792, 'loss': 2.212840859889984, 'td_loss': 0.01538949720794335, 'conservative_loss': 1.0987256860733032, 'time_step': 0.0022612261772155763} step=200
2025-07-28 15:45.25 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteCQL_20250728154525/model_200.d3


...

2025-07-28 15:45.50 [info     ] DiscreteCQL_20250728154525: epoch=99 step=9900 epoch=99 metrics={'time_sample_batch': 0.00047873735427856446, 'time_algorithm_update': 0.001869077682495117, 'loss': 2.195319347381592, 'td_loss': 0.0021575475920690224, 'conservative_loss': 1.0965808951854705, 'time_step': 0.002396836280822754} step=9900
2025-07-28 15:45.50 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteCQL_20250728154525/model_9900.d3



Epoch 100/100: 100%|██████████| 100/100 [00:00<00:00, 321.70it/s, loss=2.2, td_loss=0.00222, conservative_loss=1.1]

2025-07-28 15:45.50 [info     ] DiscreteCQL_20250728154525: epoch=100 step=10000 epoch=100 metrics={'time_sample_batch': 0.00057830810546875, 'time_algorithm_update': 0.002337493896484375, 'loss': 2.1974499917030332, 'td_loss': 0.002233420611009933, 'conservative_loss': 1.0976082861423493, 'time_step': 0.002981810569763184} step=10000


2025-07-28 15:45.50 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteCQL_20250728154525/model_10000.d3


[(1,
  {'time_sample_batch': 0.0005567145347595215,
   'time_algorithm_update': 0.0020180344581604004,
   'loss': 2.3966327929496765,
   'td_loss': 0.19853577110916376,
   'conservative_loss': 1.09904851436615,
   'time_step': 0.00263277530670166}),
 (2,
  {'time_sample_batch': 0.0004550933837890625,
   'time_algorithm_update': 0.00175459623336792,
   'loss': 2.212840859889984,
   'td_loss': 0.01538949720794335,
   'conservative_loss': 1.0987256860733032,
   'time_step': 0.0022612261772155763}),
 (3,
  {'time_sample_batch': 0.00045412778854370117,
   'time_algorithm_update': 0.0017646265029907227,
   'loss': 2.200702414512634,
   'td_loss': 0.003590560882585123,
   'conservative_loss': 1.0985559260845184,
   'time_step': 0.002267448902130127}),
 (4,
  {'time_sample_batch': 0.0005457544326782226,
   'time_algorithm_update': 0.0019359111785888672,
   'loss': 2.1985737800598146,
   'td_loss': 0.001622819603071548,
   'conservative_loss': 1.0984754824638367,
   'time_step': 0.0025356578826

### Storing and Retrieving Policy Parameters as Artifacts
Now let's store the trained policy parameters as an artifact associated with our benchmark. This demonstrates how to link artifacts to specific benchmarks for better organization and retrieval.

In [16]:
# Save the model to a temporary file
with tempfile.NamedTemporaryFile(suffix='.pt', delete=False) as temp_file:
    temp_path = temp_file.name

# Save the model using d3rlpy's save_model method
algo.save_model(temp_path)

# Read the file content as bytes
with open(temp_path, 'rb') as f:
    policy_artifact = f.read()

# Clean up the temporary file
os.unlink(temp_path)

# Create metadata for the artifact, linking it to our benchmark
policy_metadata = ArtifactMetadata(
    name='trained_cql_policy', 
    description='Trained CQL policy parameters for MountainCar environment',
    benchmark_id=loaded_tupli_env.id  # Link artifact to the benchmark
)

# Store the artifact
stored_policy = storage.store_artifact(artifact=policy_artifact, metadata=policy_metadata)
print(f"Stored policy artifact with ID: {stored_policy.id}")
print(f"Artifact linked to benchmark: {stored_policy.benchmark_id}")

Stored policy artifact with ID: 48ecc6b86e9a4cdeafd000e293511a6b
Artifact linked to benchmark: 7b334ed6b7724bec8872418ad32667a4


Another collaborator could now retrieve this artifact by listing all artifacts associated to our benchmark and downloading it. 

In [17]:
# Create a filter to find artifacts associated with our benchmark
benchmark_filter = FilterEQ(key='benchmark_id', value=loaded_tupli_env.id)

# List all artifacts associated with this benchmark
benchmark_artifacts = storage.list_artifacts(filter=benchmark_filter)

print(f"Found {len(benchmark_artifacts)} artifacts associated with benchmark {loaded_tupli_env.id}:")
for artifact in benchmark_artifacts:
    print(f"  - ID: {artifact.id}")
    print(f"    Name: {artifact.name}")
    print(f"    Description: {artifact.description}")
    print(f"    Benchmark ID: {artifact.benchmark_id}")
    print(f"    Created: {artifact.created_at}")
    print()

Found 0 artifacts associated with benchmark 4b076cdedc4d433ea4da6d2801c05ca0:


Finally, we demonstrate deserialization of the stored policy.

In [18]:
# Load the policy artifact
loaded_policy_artifact = storage.load_artifact(stored_policy.id)

# Write the bytes to a temporary file
with tempfile.NamedTemporaryFile(suffix='.pt', delete=False) as temp_file:
    temp_path = temp_file.name
    temp_file.write(loaded_policy_artifact)

# Create a new algorithm instance and load the model
loaded_algo = DiscreteCQLConfig().create(device='cpu')
loaded_algo.build_with_env(loaded_tupli_env)
loaded_algo.load_model(temp_path)

print("Successfully loaded trained CQL policy!")


Successfully loaded trained CQL policy!


### Testing the Trained Policy
Now, let us test the trained (and loaded) policy:

In [19]:
# activate rendering
setattr(loaded_tupli_env.unwrapped, 'render_mode', 'human')
# deactivate recording of episodes
loaded_tupli_env.deactivate_recording()
# run the environment
np.random.seed(seed=42)
obs, info = loaded_tupli_env.reset(seed=42)

for step in range(800):
    action = np.int64(loaded_algo.predict(np.expand_dims(obs, axis=0))[0])
    obs, reward, done, truncated, info = loaded_tupli_env.step(action)
    if done or truncated:
        print(f'Episode finished after {step + 1} timesteps')
        obs, info = loaded_tupli_env.reset()
# deactivate rendering
loaded_tupli_env.close()

Episode finished after 375 timesteps
Episode finished after 694 timesteps


The trained policy manages to reach the flag even though it has only learned from random actions!

### Deleting Benchmarks
To clean up our storage, we now delete the benchmark and all related artifacts. Episodes will automatically be deleted, too.

In [20]:
# Clean up: First delete the policy artifact we created
print(f"Deleting policy artifact: {stored_policy.id}")
storage.delete_artifact(stored_policy.id)

# Then delete the benchmark and all remaining related artifacts
# Episodes will automatically be deleted too
print(f"Deleting benchmark: {loaded_tupli_env.id}")
loaded_tupli_env.delete(delete_artifacts=True)

print("Cleanup completed!")

Deleting policy artifact: 48ecc6b86e9a4cdeafd000e293511a6b
Deleting benchmark: 4b076cdedc4d433ea4da6d2801c05ca0
Cleanup completed!
